In [1]:
#! /data/zhangxingyu/anaconda3/envs/py38/bin/python
import argparse
import pandas as pd
from tqdm import tqdm
import yaml
import math
import numpy as np
from datetime import datetime
import json
import bisect

config = yaml.load(open('../config/config.yaml'))
exception_name_columns = config['exception_name']['train']

In [4]:
nc_exceptions_train = pd.read_csv('../'+config['data']['features']['train']['n_exceptions'])
nc_info_train = pd.read_csv('../'+config['data']['raw']['train']['nc_info'])
nc_exceptions_test = pd.read_csv('../'+config['data']['features']['test']['n_exceptions'])
nc_info_test = pd.read_csv('../'+config['data']['raw']['test']['nc_info'])

In [5]:
nc_info_train

,nc_ip,physical_model,cpu_generation_full,manufacturer,cluster_alias,cluster_usage,mem_pn,mem_manufacturer,bios_version,os_release,bmc_version
0,79551bf1-e0de-4d88-b820-b80fb18a0841,physical_model_70,cpu_generation_full_5,manufacturer_4,79c0fb8e-a630-47db-9ca8-65e20d408ef6,cluster_usage_15,mem_pn_12,mem_manufacturer_6,bios_version_28,os_release_8,bmc_version_94
1,415af949-2c36-4d65-9f80-7c89c515eabc,physical_model_10,cpu_generation_full_3,manufacturer_3,6634bc41-9f95-4b9b-bcb8-8813a96d0f73,cluster_usage_15,mem_pn_8,mem_manufacturer_6,bios_version_112,os_release_8,bmc_version_85
2,29c8f397-5ccb-4628-adf4-2a99eea68cc1,physical_model_10,cpu_generation_full_3,manufacturer_4,4982d1a7-5535-46b3-83b1-70ea1b4d8b95,cluster_usage_15,mem_pn_54,mem_manufacturer_2,bios_version_64,os_release_8,bmc_version_24
3,669814be-f240-488c-a50c-d1feedab8d4e,physical_model_10,cpu_generation_full_3,manufacturer_4,4982d1a7-5535-46b3-83b1-70ea1b4d8b95,cluster_usage_15,mem_pn_54,mem_manufacturer_2,bios_version_64,os_release_8,bmc_version_24
4,34ab5ccb-a592-4c2a-82b7-e8c83827959a,physical_model_64,cpu_generation_full_3,manufacturer_4,5b5a82b7-188e-44c1-acee-78591115680e,cluster_usage_15,mem_pn_16,mem_manufacturer_6,bios_version_16,os_release_8,bmc_version_42
...,...,...,...,...,...,...,...,...,...,...,...
14537,2de4bd7a-7e4e-4e2f-8376-c89994384c44,physical_model_10,cpu_generation_full_3,manufacturer_8,69c2b107-b68f-45b1-a497-3fd9776eab56,cluster_usage_15,mem_pn_8,mem_manufacturer_6,bios_version_99,os_release_8,bmc_version_95
14538,14736c5d-1a37-4248-8151-699b9fccbcf1,physical_model_33,cpu_generation_full_3,manufacturer_8,65e6ac5e-5591-44b4-91c2-6f0a11534558,cluster_usage_15,mem_pn_26,mem_manufacturer_4,bios_version_99,os_release_8,bmc_version_95
14539,ac76fad6-7e52-4bb7-964d-e0a705bd5e8e,physical_model_10,cpu_generation_full_3,manufacturer_3,8668b493-f3c4-4057-a979-96cf299593b8,cluster_usage_10,mem_pn_8,mem_manufacturer_6,bios_version_123,os_release_8,bmc_version_99
14540,f5fc55cf-0966-4176-bf7d-302967eafa83,physical_model_14,cpu_generation_full_5,manufacturer_8,8668b493-f3c4-4057-a979-96cf299593b8,cluster_usage_10,mem_pn_19,mem_manufacturer_2,bios_version_164,os_release_8,bmc_version_94


In [8]:
nc_sample_label.join(nc_exceptions_test.set_index('nc_ip'), on='nc_ip', how='inner')

,nc_ip,sample_time,nc_down_label,exception_name_10,exception_name_11,exception_name_12,exception_name_14,exception_name_15,exception_name_16,exception_name_17,...,exception_name_55,exception_name_56,exception_name_57,exception_name_58,exception_name_59,exception_name_6,exception_name_60,exception_name_7,exception_name_8,exception_name_9
0,dccf231b-5b94-4763-9f07-232fd365929e,2017-01-20 15:25:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,"['2017-01-20 15:23:00', '2017-01-20 15:23:00',...",NaN,NaN,NaN,NaN,NaN,NaN
1,7f515357-2f44-4513-a554-de0e06245160,2017-03-28 15:25:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,"['2017-03-28 15:22:57', '2017-03-28 15:22:57',...",NaN,NaN,NaN,NaN,NaN,NaN
2,d2657067-9a4b-49ac-8287-5f14295d903a,2016-04-09 15:25:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,"['2016-04-09 15:23:07', '2016-04-09 15:23:07',...",NaN,NaN,NaN,NaN,NaN,NaN
3,7cd5b33e-a152-481a-95d9-3601a6137622,2020-03-28 15:25:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,"['2020-03-28 15:22:46', '2020-03-28 15:22:46',...",NaN,NaN,NaN,NaN,NaN,NaN
4,0bfc7443-2527-4fd5-9669-6acb7643397c,2016-04-23 16:30:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,"['2016-04-23 16:29:24', '2016-04-23 16:29:24',...",NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
146145,f8553f4b-03a7-4cc6-b6f6-8d52e1eed2f6,2020-02-27 17:40:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,"['2020-02-27 17:39:10', '2020-02-27 17:39:10',...",NaN,NaN,NaN,NaN,NaN,NaN
146146,de556185-ee86-410e-9c9d-ed7059650a76,2019-04-21 15:15:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,"['2019-04-21 15:14:16', '2019-04-21 15:14:16',...",NaN,NaN,NaN,NaN,NaN,NaN
146147,da9720c1-6797-49ec-8b55-b0d9b9b7394a,2019-12-23 11:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,"['2019-12-23 10:59:19', '2019-12-23 10:59:19',...",NaN,NaN,NaN,NaN,NaN,NaN
146148,fb96a605-1aad-40ed-aeee-c30211bfacb3,2016-02-16 00:45:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,"['2016-02-16 00:40:22', '2016-02-16 00:40:22',...",NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
for exception_name in exception_name_columns:
    nc_exceptions_train[exception_name] = nc_exceptions_train[exception_name].fillna('[]').apply(eval)
    nc_exceptions_test[exception_name] = nc_exceptions_test[exception_name].fillna('[]').apply(eval)


Error: Session cannot generate requests

In [ ]:
nc_exceptions_train

Error: Session cannot generate requests

In [33]:
nc_sample_label_with_exceptions = nc_sample_label.join(nc_exceptions_train.set_index('nc_ip'), how='left', on='nc_ip').fillna('')
str2date = lambda x: datetime.strptime(x, "%Y-%m-%d %H:%M:%S.%f") if '.' in x else datetime.strptime(x, "%Y-%m-%d %H:%M:%S")
series2date = lambda x: pd.to_datetime(x, format='%Y-%m-%d %H:%M:%S')
date2str = lambda x: x.strftime("%Y-%m-%d %H:%M:%S.%f")

nc_sample_label_with_exceptions['sample_time'] = series2date(nc_sample_label_with_exceptions['sample_time'])
nc_sample_label_with_exceptions['start_time'] = nc_sample_label_with_exceptions['sample_time'] - pd.DateOffset(days=days)


In [40]:
# for exception_name in exception_name_columns:
    # nc_exceptions_train[exception_name] = nc_exceptions_train[exception_name].fillna('[]').apply(eval)
    # nc_exceptions_train[exception_name] = nc_exceptions_train[exception_name].fillna('[]').apply(lambda x: x.strip("[]").split(", "))
    # nc_exceptions_test[exception_name] = nc_exceptions_test[exception_name].fillna('[]').apply(lambda x: x.strip("[]").split(", "))
    # nc_sample_label_with_exceptions[exception_name] = nc_sample_label_with_exceptions[exception_name].fillna('[]').apply(eval)

In [31]:
def get_date_cnt_from_intervals(start_date, end_date, date_list):
    if date_list == '': return 0
    return bisect.bisect_left(date_list, end_date.str) - bisect.bisect_left(date_list, start_date)


In [40]:
for exception_name in tqdm(exception_name_columns, total=len(exception_name_columns)):
    exception_cnt_name = exception_name + '_cnt' 
    nc_sample_label_with_exceptions[exception_cnt_name] = nc_sample_label_with_exceptions.apply(lambda x: get_date_cnt_from_intervals(date2str(x['start_time']), date2str(x['sample_time']), x[exception_name]), axis=1)

100%|██████████| 57/57 [11:25<00:00, 12.02s/it]


In [42]:
nc_sample_label_with_exceptions[['nc_ip', 'sample_time', 'nc_down_label'] + [exception_name + '_cnt' for exception_name in exception_name_columns]].to_csv('../'+config['data']['features']['train']['n_exceptions'], index=False)

FileNotFoundError: [Errno 2] No such file or directory: 'data/features/ecsmpf_round1_train_20210625/round1_train_n_exceptions.csv'

In [7]:

res_tuples = []
for i in tqdm(nc_sample_label_with_exceptions.index, total=nc_sample_label_with_exceptions.shape[0]):
    row = nc_sample_label_with_exceptions.loc[i]
    # new_tuple = [row['nc_ip'], row['sample_time'], row['nc_down_label']]\
    #     + [len([1 for date_str in row[exception_name] if date_str and row['start_time'] <= str2date(date_str) < row['sample_time']]) for exception_name in exception_name_columns]
    # res_tuples.append(new_tuple)
    pass
res_columns = ['nc_ip', 'sample_time', 'nc_down_label'] + exception_name_columns
res_tbl = pd.DataFrame(res_tuples, columns=res_columns)


 53%|█████▎    | 281770/530922 [00:35<00:31, 8016.84it/s]


KeyboardInterrupt: 

SyntaxError: invalid syntax (<ipython-input-5-bd5c75f13f21>, line 12)

,nc_ip,sample_time,nc_down_label,exception_name_10,exception_name_11,exception_name_12,exception_name_14,exception_name_15,exception_name_16,exception_name_17,...,exception_name_55,exception_name_56,exception_name_57,exception_name_58,exception_name_59,exception_name_6,exception_name_60,exception_name_7,exception_name_8,exception_name_9
0,f4724808-d7ec-4cad-9179-29414b1188b0,2020-12-19 15:25:00,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,"['2020-12-19 15:23:08', '2020-12-19 15:23:08',...",NaN,NaN,NaN,NaN,NaN,NaN
1,e5336393-ecca-443d-8df8-f09e928c8f00,2017-09-09 15:25:00,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,"['2017-09-09 15:23:00', '2017-09-09 15:23:00',...",NaN,NaN,NaN,NaN,NaN,NaN
2,30d42ae0-76e3-4121-9098-0a3557cc8a59,2016-06-17 15:25:00,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,"['2016-06-17 15:22:50', '2016-06-17 15:22:50',...",NaN,NaN,NaN,NaN,NaN,NaN
3,88226889-3382-4f0d-8f4f-600e67c9111f,2018-07-17 15:25:00,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,"['2018-07-17 15:22:57', '2018-07-17 15:22:57',...",NaN,NaN,NaN,NaN,NaN,NaN
4,c2e430d3-732f-4c0e-b740-92d01195fba6,2017-11-22 17:55:00,False,['2017-11-23 09:19:29'],NaN,NaN,NaN,NaN,"['2017-11-22 17:52:16.810000', '2017-11-23 09:...","['2017-11-22 17:52:16', '2017-12-08 09:41:27',...",...,NaN,NaN,"['2017-11-22 17:52:16', '2017-11-23 09:19:29',...","['2017-11-20 15:22:54', '2017-11-20 15:22:54',...",NaN,NaN,NaN,NaN,NaN,['2017-11-23 09:19:29']
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
530917,f55ea661-dc45-4056-bff6-5ab732af8a04,2017-12-17 12:10:00,False,NaN,NaN,NaN,NaN,"['2017-12-08 14:10:19', '2018-01-15 05:28:02',...","['2017-11-09 20:48:54.617000', '2017-11-10 20:...","['2017-11-09 20:48:54', '2017-11-10 20:35:09',...",...,NaN,NaN,"['2017-11-09 20:48:54', '2017-11-10 20:35:09',...","['2017-11-09 20:48:54', '2017-11-09 20:48:54',...",NaN,"['2017-12-08 14:10:19.273000', '2018-01-29 02:...",NaN,NaN,NaN,"['2017-12-07 14:24:04', '2018-01-14 05:41:46',..."
530918,f55ea661-dc45-4056-bff6-5ab732af8a04,2017-12-28 09:40:00,False,NaN,NaN,NaN,NaN,"['2017-12-08 14:10:19', '2018-01-15 05:28:02',...","['2017-11-09 20:48:54.617000', '2017-11-10 20:...","['2017-11-09 20:48:54', '2017-11-10 20:35:09',...",...,NaN,NaN,"['2017-11-09 20:48:54', '2017-11-10 20:35:09',...","['2017-11-09 20:48:54', '2017-11-09 20:48:54',...",NaN,"['2017-12-08 14:10:19.273000', '2018-01-29 02:...",NaN,NaN,NaN,"['2017-12-07 14:24:04', '2018-01-14 05:41:46',..."
530919,f55ea661-dc45-4056-bff6-5ab732af8a04,2018-01-03 08:15:00,False,NaN,NaN,NaN,NaN,"['2017-12-08 14:10:19', '2018-01-15 05:28:02',...","['2017-11-09 20:48:54.617000', '2017-11-10 20:...","['2017-11-09 20:48:54', '2017-11-10 20:35:09',...",...,NaN,NaN,"['2017-11-09 20:48:54', '2017-11-10 20:35:09',...","['2017-11-09 20:48:54', '2017-11-09 20:48:54',...",NaN,"['2017-12-08 14:10:19.273000', '2018-01-29 02:...",NaN,NaN,NaN,"['2017-12-07 14:24:04', '2018-01-14 05:41:46',..."
530920,f55ea661-dc45-4056-bff6-5ab732af8a04,2018-01-18 04:50:00,False,NaN,NaN,NaN,NaN,"['2017-12-08 14:10:19', '2018-01-15 05:28:02',...","['2017-11-09 20:48:54.617000', '2017-11-10 20:...","['2017-11-09 20:48:54', '2017-11-10 20:35:09',...",...,NaN,NaN,"['2017-11-09 20:48:54', '2017-11-10 20:35:09',...","['2017-11-09 20:48:54', '2017-11-09 20:48:54',...",NaN,"['2017-12-08 14:10:19.273000', '2018-01-29 02:...",NaN,NaN,NaN,"['2017-12-07 14:24:04', '2018-01-14 05:41:46',..."
